In [17]:
import torch
import os
import pandas as pd

In [44]:
def load_data(start_year=2002,end_year=2017):
    col_name = ["date","time","open","high","low","close","volume"]
    df_full = pd.DataFrame(columns = col_name)
    for root, dirs, files in os.walk("/stock_prediction/stock_prediction/EURGBP/"):
        for file in files:
            year = int(file.split(".")[0].split("_")[-1])
            if (file.endswith(".csv") and year >= start_year 
            and year <= end_year):
                df = pd.read_csv(os.path.join(root,file),names = col_name)
                df_full = pd.concat([df_full,df])
                print("Stock data of year {0} loaded".format(str(year)))
    return df_full